In [ ]:
from mine import trend_utils
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from utils import date_utils
import ccxt

df1 = pd.read_csv('data/20200101-ETH_future.csv')
binance = ccxt.binance()
eth = binance.fetch_ticker("ETH/USDT")

a = trend_utils.trend(df1[len(df1)-34:len(df1)].reset_index(drop=True))
print(a.near_trend(eth['close']))
a.graph(df1)



In [ ]:
#data fetching
from utils import ccxt_utils
import ccxt
from mine import market
import pandas as pd

COLUMNS  = ['datetime', 'open', 'high', 'low', 'close', 'volume']

binance= ccxt.binance({
    'options': {
        'adjustForTimeDifference': True,
        'defaultType': 'future'
    },
    'rateLimit': 10000,
    'enableRateLimit': True,
    # 'verbose': True,
    'apiKey': 'BpLnIGqKodX0EXAxuR14cpbYax4T6Z1KvrQ8Xbevk3PdmcMjO3J06UK9g6IDS1eG',
    'secret': '90uKFUV4b0HBjOLtmQmSsHdv9rrYGcZlufEwfrnryAfHzT2BB3Xck5ufsfQXnfW9',
})

since = '2021-01-01 00:00:00'
since_timestamp = binance.parse8601(since)
print(since_timestamp)
BINANCE_SYMBOL='ETH/USDT'

data_1h = ccxt_utils.fetch_ohlcv(BINANCE_SYMBOL, binance, timeframe='1h', start=since, end=None)
data_1m = ccxt_utils.fetch_ohlcv(BINANCE_SYMBOL, binance, timeframe='1m', start=since, end=None)

data_1h_df = ccxt_utils.ohlcv_to_df(data_1h['data'])
data_1h_df.to_csv('data/20200101-ETH_future.csv')
data_1m_df = ccxt_utils.ohlcv_to_df(data_1m['data'])
data_1m_df.to_csv('data/20200101-ETH_future_1m.csv')


In [1]:
#test
from mine import trend_utils
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from utils import date_utils
import ccxt

df1 = pd.read_csv('data/20200101-binance_futures-1h.csv')


columns=['datetime', 'buy/sell', 'profit']
temp='none'
temp_price=df1['open'][0]
total_profit=500000
buy_price=0
sell_price=0
df2 = pd.read_csv('data/20200101-binance_futures-1m.csv')
for i in range(21, len(df1)):
    data=[]
    d=df1[i-20:i].reset_index(drop=True)
    a=trend_utils.trend(d, 20)
    for j in range(60*i, 60*i+60):
        temp_price=df2['close'][j]
        position=a.near_trend(temp_price, date_utils.str_to_utctimestamp(df2['datetime'][j]))
        
        if buy_price>0 and temp_price<=buy_price*0.95:
            print("청산")
            total_profit=0
        if sell_price>0 and temp_price>=sell_price*1.05:
            print("청산")
            total_profit=0
            
        if position == 'buy' and temp=='none':
            buy_price=df2['close'][j]
            data.append([df2['datetime'][j], 'buy', '-'])
            temp='buy'
        elif position == 'sell' and temp=='none':
            sell_price=df2['close'][j]
            data.append([df2['datetime'][j], 'sell', '-'])
            temp='sell'
        elif position == 'buy' and temp=='sell':
            buy_price=df2['close'][j]
            total_profit=total_profit*(sell_price/df2['close'][j]*20-19)*0.999925
            sell_price=0
            data.append([df2['datetime'][j], 'buy', '-'])
            temp='buy'
        elif position == 'sell' and temp=='buy':
            sell_price=df2['close'][j]
            total_profit=total_profit*(df2['close'][j]/buy_price*20-19)*0.999925
            buy_price=0
            data.append([df2['datetime'][j], 'sell', '-'])
            temp='sell'
        elif position == 'none' and temp=='sell':
            temp='none'
            total_profit=total_profit*(sell_price/df2['close'][j]*20-19)*0.999925
            data.append([df2['datetime'][j], 'none', total_profit])
            sell_price=0
        elif position == 'none' and temp=='buy':
            temp='none'
            total_profit=total_profit*(df2['close'][j]/buy_price*20-19)*0.999925
            data.append([df2['datetime'][j], 'none', total_profit])
            buy_price=0
    record=pd.DataFrame(data, columns=columns)
    record.to_csv("./data/test.csv", mode='a')
data=[['total', '',total_profit]]
record=pd.DataFrame(data, columns=columns)
record.to_csv("./data/test.csv", mode='a')
    

In [ ]:
#최적화

from mine import market, trend_utils
import pandas
from mine import trend_utils
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from utils import date_utils
import ccxt

df1 = pd.read_csv('data/20200101-binance_futures-1h.csv')
df2 = pd.read_csv('data/20200101-binance_futures-1m.csv')
columns=['size', 'profit']
data=[]
size=20
leverage=20
while leverage :
    total_profit=100
    buy_price=0
    sell_price=0
    temp='none'
    for i in range(size+1, len(df1)):
        d=df1.iloc[i-size:i].reset_index(drop=True)
        a=trend_utils.trend(d, size)
        for j in range(60*i, 60*i+60):
            temp_price=df2['close'][j]
            position=a.near_trend(temp_price, date_utils.str_to_utctimestamp(df2['datetime'][j]))
            
            #leverage에 따른 청산가능성
            if buy_price>0 and temp_price<=buy_price*(leverage-1)/leverage:
                print("청산")
                total_profit=0
            if sell_price>0 and temp_price>=sell_price*(leverage+1)/leverage:
                print("청산")
                total_profit=0

            #매매수익 계산
            if position == 'buy' and temp=='none':
                buy_price=df2['close'][j]
                temp='buy'
            elif position == 'sell' and temp=='none':
                sell_price=df2['close'][j]
                temp='sell'
            elif position == 'buy' and temp=='sell':
                buy_price=df2['close'][j]
                total_profit=total_profit*(sell_price/df2['close'][j]*leverage-leverage+1)*0.99925
                sell_price=0
                temp='buy'
            elif position == 'sell' and temp=='buy':
                sell_price=df2['close'][j]
                total_profit=total_profit*(df2['close'][j]/buy_price*leverage-leverage+1)*0.99925
                buy_price=0
                temp='sell'
            elif position == 'none' and temp=='sell':
                temp='none'
                total_profit=total_profit*(sell_price/df2['close'][j]*leverage-leverage+1)*0.99925
                sell_price=0
            elif position == 'none' and temp=='buy':
                temp='none'
                total_profit=total_profit*(df2['close'][j]/buy_price*leverage-leverage+1)*0.99925
                buy_price=0
    
    data=[size, total_profit]
    #data=[leverage, total_profit]
    print(data)
    record=pd.DataFrame([data], columns=columns)
    record.to_csv("./data/test.csv", mode='a')
    size-=1
    #leverage+=1